# Project 7 

In this project, you will implement the the clustering techniques that you've learned this week.

## Step 1: Load the python libraries that you will need for this project 

In [818]:
import pandas as pd 
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk 
import sqlalchemy
import patsy
import seaborn as sns
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import FunctionTransformer, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.decomposition import PCA
from sklearn.base import TransformerMixin
from sklearn.grid_search import GridSearchCV

%matplotlib inline

## Step 2: Load/Examine your data 

In [819]:
df_raw = pd.read_csv("../assets/airport_cancellations.csv")
df = df_raw.dropna() 
print df.head()

  Airport    Year  Departure Cancellations  Arrival Cancellations  \
0     ABQ  2004.0                    242.0                  235.0   
1     ABQ  2005.0                    221.0                  190.0   
2     ABQ  2006.0                    392.0                  329.0   
3     ABQ  2007.0                    366.0                  304.0   
4     ABQ  2008.0                    333.0                  300.0   

   Departure Diversions  Arrival Diversions  
0                  71.0                46.0  
1                  61.0                33.0  
2                  71.0               124.0  
3                 107.0                45.0  
4                  79.0                42.0  


In [820]:
df_raw2 = pd.read_csv("../assets/airports.csv")
df2 = df_raw2.dropna() 
print df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3498 entries, 0 to 5163
Data columns (total 13 columns):
Key                        3498 non-null float64
LocID                      3498 non-null object
AP_NAME                    3498 non-null object
ALIAS                      3498 non-null object
Facility Type              3498 non-null object
FAA REGION                 3498 non-null object
COUNTY                     3498 non-null object
CITY                       3498 non-null object
STATE                      3498 non-null object
AP Type                    3498 non-null object
Latitude                   3498 non-null float64
Longitude                  3498 non-null float64
Boundary Data Available    3498 non-null object
dtypes: float64(3), object(10)
memory usage: 382.6+ KB
None


In [821]:
df_raw3 = pd.read_csv("../assets/Airport_operations.csv")
df3 = df_raw3.dropna() 
print df3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 841 entries, 0 to 840
Data columns (total 15 columns):
airport                               841 non-null object
year                                  841 non-null int64
departures for metric computation     841 non-null int64
arrivals for metric computation       841 non-null int64
percent on-time gate departures       841 non-null float64
percent on-time airport departures    841 non-null float64
percent on-time gate arrivals         841 non-null float64
average_gate_departure_delay          841 non-null float64
average_taxi_out_time                 841 non-null float64
average taxi out delay                841 non-null float64
average airport departure delay       841 non-null float64
average airborne delay                841 non-null float64
average taxi in delay                 841 non-null float64
average block delay                   841 non-null float64
average gate arrival delay            841 non-null float64
dtypes: float64(1

## Step 3: Problem Statement

## Step 4: Store Data in PostGreSQL

Get user logged in and connect to default postgres db using that user

In [822]:
user = !id -un
print user
!rm /usr/local/var/postgres/postmaster.pid
!brew services restart postgresql
engine = sqlalchemy.create_engine('postgresql://' + user[0] + '@localhost:5432/postgres')

['alexandertam']
==> Successfully started `postgresql` (label: homebrew.mxcl.postgresql)


convert columns to lowercase. Makes its easier to store dataframes in postgres

In [823]:
df.columns = map(str.lower, df.columns)
df2.columns = map(str.lower, df2.columns)
df3.columns = map(str.lower, df3.columns)

add dataframes as table to database, replace if they already exist

In [824]:
df.to_sql("airport_cancellations",con = engine, if_exists = "replace")

In [825]:
df2.to_sql("airports", con = engine, if_exists = "replace")

In [826]:
df3.to_sql("airport_operations", con = engine, if_exists="replace")

get the data expect to use for Principal Component Analysis

In [827]:
joined = pd.read_sql('SELECT o.*  FROM airports as a JOIN airport_cancellations as ac ON a.locid=ac.airport JOIN airport_operations as o ON a.locid=o.airport;',con = engine)

get all the data

In [828]:
everything = pd.read_sql('SELECT * FROM airports as a INNER JOIN airport_cancellations as ac ON a.locid=ac.airport INNER JOIN airport_operations as o ON a.locid=o.airport;',con = engine)

close the db connection

In [829]:
!brew services stop postgresql

Stopping `postgresql`... (might take a while)
==> Successfully stopped `postgresql` (label: homebrew.mxcl.postgresql)


save "everything" as a csv for use in Tableau later

In [830]:
everything.to_csv("../assets/everything.csv",encoding="utf-8")

#### 1.2 What are the risks and assumptions of our data? 

## Step 5: Clean the Data

copy data to new var, so I don't have to rerun the whole file. dropna() and index column

In [831]:
data = joined.copy(deep=True)
data = joined.dropna()
del data["index"]
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8789 entries, 0 to 8788
Data columns (total 15 columns):
airport                               8789 non-null object
year                                  8789 non-null int64
departures for metric computation     8789 non-null int64
arrivals for metric computation       8789 non-null int64
percent on-time gate departures       8789 non-null float64
percent on-time airport departures    8789 non-null float64
percent on-time gate arrivals         8789 non-null float64
average_gate_departure_delay          8789 non-null float64
average_taxi_out_time                 8789 non-null float64
average taxi out delay                8789 non-null float64
average airport departure delay       8789 non-null float64
average airborne delay                8789 non-null float64
average taxi in delay                 8789 non-null float64
average block delay                   8789 non-null float64
average gate arrival delay            8789 non-null float64


generate dummy variables for airports and generate new dataframe for dummis

In [832]:
df = data
categories = ["airport"]
for category in categories:
    series = df[category]
    dummies = pd.get_dummies(series, prefix=category)
    df = pd.concat([df, dummies], axis=1)
dataWithDummies = df.copy(deep=True)

del dataWithDummies["airport"]
del data["airport"]

## Step 6: Run PCA

##### Defining custom transformers for Sklearn pipeline, ended up not using them. Keep for later use

In [859]:
class MultiColumnLabelEncoder(TransformerMixin):
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = pd.DataFrame(X,columns=self.columns)
        #print output.head()
        if self.columns is not None:
            for col in self.columns:
                output[col] = LabelEncoder().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = LabelEncoder().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [834]:
#test MultiColumnLabelEncoder class works

#mcle = MultiColumnLabelEncoder(columns= columnsToLabelEncode)
#mcle.fit_transform(joined)

In [835]:
# class MultiColumnMinMaxScaler(TransformerMixin):
#     def __init__(self,columns = None):
#         self.columns = columns # array of column names to encode

#     def fit(self,X,y=None):
#         return self # not relevant here

#     def transform(self,X):
#         '''
#         Transforms columns of X specified in self.columns using
#         LabelEncoder(). If no columns specified, transforms all
#         columns in X.
#         '''
#         output = X.copy()
#         if self.columns is not None:
#             for col in self.columns:
#                 output[col] = pre.MinMaxScaler().fit_transform(output[col].reshape(-1,1))
#         else:
#             for colname,col in output.iteritems():
#                 output[colname] = pre.MinMaxScaler().fit_transform(col)
#         return output

#     def fit_transform(self,X,y=None):
#         return self.fit(X,y).transform(X)

In [836]:
#test MultiColumnMinMaxScaler class works
#mcmms = MultiColumnMinMaxScaler(columns= columnsToScale)
#mcmms.fit_transform(joined).head()

In [837]:
# columnsToLabelEncode = ["airport"]
# columnsToRemove = ["locid","index","key","boundary data available"]
# columnsToScale = ["year"]
# columnsToSelect = ['airport','year','departure cancellations','arrival cancellations']

In [838]:
#data = data[[x for x in data.columns.values.tolist() if x not in columnsToRemove]]
#joined = joined[columnsToSelect].copy(deep=True)

In [839]:
# pipe = make_pipeline(
#     MultiColumnLabelEncoder(columns = columnsToLabelEncode),
#     MultiColumnMinMaxScaler(columns = columnsToScale))
# result1 = pipe.fit_transform(data)

#### Run PCA

This set of data contains all the columns from the operations table + the airports as dummies

In [840]:
dummyColumns = dataWithDummies.columns

pca = PCA()

pipe = Pipeline([('pca', pca)])

tmp = np.array(dataWithDummies)
dataOriginal = pipe.fit_transform(tmp)
dataOriginal = pd.DataFrame(dataOriginal,columns=dummyColumns)
dataOriginal.head()

,year,departures for metric computation,arrivals for metric computation,percent on-time gate departures,percent on-time airport departures,percent on-time gate arrivals,average_gate_departure_delay,average_taxi_out_time,average taxi out delay,average airport departure delay,...,airport_SFO,airport_SJC,airport_SJU,airport_SLC,airport_SMF,airport_SNA,airport_STL,airport_SWF,airport_TPA,airport_TUS
0,83911.691513,-70.213016,-4.649349,4.951224,1.811950,0.854728,-0.378470,-0.701735,-0.210493,0.038660,...,0.009102,0.005251,0.026531,-0.019879,-0.010008,0.002062,0.002999,0.004427,-0.002847,-5.238288e-15
1,86799.061101,-207.114916,-5.870731,4.145722,1.120552,0.690475,-0.401350,-0.777936,-0.225447,0.126866,...,0.017122,0.024540,0.004525,-0.007613,-0.015892,0.001120,-0.001101,0.003245,-0.002261,-9.386819e-15
2,88798.515025,-1242.235830,-3.366770,3.074479,1.641045,0.074002,-0.417327,-0.526339,-0.120900,0.008117,...,-0.010165,-0.025269,-0.011938,-0.030223,-0.012347,0.000164,-0.001786,0.000535,0.000368,-7.366389e-15
3,84565.001583,-428.947670,-2.523626,2.033138,1.476709,-0.453662,-0.349287,-0.708434,0.187694,0.269402,...,-0.035472,-0.040785,-0.024050,-0.045301,-0.005106,-0.003326,0.007214,0.001151,0.002890,-2.194979e-14
4,90055.122443,-112.831021,-3.823615,1.136024,0.615505,0.074235,-0.567142,-0.687876,0.158743,0.011751,...,-0.048355,-0.013538,-0.017466,-0.059261,0.005612,-0.001326,0.012261,0.003350,0.000985,-6.089294e-15


The data set above is the PCA transformation of the original fields. We can see year is much larger that all other fields. The airports dummy columns also very very small. We may remove them, but lets try applying standard scaling first, to put the numbers across columns on the same scale.

Here we calculate the explained variance as a percentage to see which PCA has the greatest effect

In [841]:
sumExplainedVariance = sum(pca.explained_variance_)
varianceExplained = [(i / sumExplainedVariance)*100 for i in sorted(pca.explained_variance_, reverse=True)]
cumulativeVarianceexplained = np.cumsum(varianceExplained)

In [842]:
varianceExplained

[99.990354123878731,
 0.0096455617420605939,
 1.960411107919446e-07,
 5.4350140117527057e-08,
 4.0056860437285544e-08,
 9.4849260988049852e-09,
 3.5474113398346181e-09,
 3.5375947025362688e-09,
 1.721070621914049e-09,
 8.4784670102669627e-10,
 1.0511843260825852e-10,
 7.510664195174607e-11,
 7.2120850023789086e-11,
 7.19613087086532e-11,
 7.192206303948348e-11,
 7.1922063039481567e-11,
 7.1922063039481567e-11,
 7.1922063039481554e-11,
 7.1922063039481554e-11,
 7.1922063039481554e-11,
 7.1922063039481554e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.1922063039481528e-11,
 7.19220630

In [843]:
cumulativeVarianceexplained

array([ 99.99035412,  99.99999969,  99.99999988,  99.99999994,
        99.99999998,  99.99999999,  99.99999999,  99.99999999,
        99.99999999, 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.        ,
       100.        , 100.        , 100.        , 100.  

Frame varianceExplained and cumulativeVarianceExplained we can see that 99.99 percent of the variance is explained by one eigenvalue. We want some variance to be spread across other eigenvalues, so we can identify the effects of dimensions on flight cancellations. The dummy airport variables may have very little effect.

### Lets try the same thing with Standard Scaling

In [844]:
dummyColumns = dataWithDummies.columns

pca = PCA()
ss = StandardScaler()

pipe = Pipeline([('ss', ss),('pca', pca)])

tmp = np.array(dataWithDummies)
dataScaled = pipe.fit_transform(tmp)
dataScaled = pd.DataFrame(dataScaled,columns=dummyColumns)
dataScaled.head()

,year,departures for metric computation,arrivals for metric computation,percent on-time gate departures,percent on-time airport departures,percent on-time gate arrivals,average_gate_departure_delay,average_taxi_out_time,average taxi out delay,average airport departure delay,...,airport_SFO,airport_SJC,airport_SJU,airport_SLC,airport_SMF,airport_SNA,airport_STL,airport_SWF,airport_TPA,airport_TUS
0,2.558976,-0.138574,-0.236248,-0.224405,-0.242590,0.198879,-1.459025,-0.176045,-1.357238,0.219374,...,-0.031757,0.073803,0.064583,-0.029618,0.009556,-0.050062,-0.010719,0.031347,-0.002511,9.531575e-16
1,3.010396,0.098281,-0.143537,-0.280649,-0.054926,0.199138,-1.382507,-0.203433,-1.235267,0.128743,...,-0.007957,0.045635,-0.028351,-0.023695,0.017146,-0.050291,-0.001016,0.031415,-0.001095,1.429035e-14
2,2.410114,-0.487076,0.023867,-0.114580,0.079128,0.212893,-1.260966,-0.179098,-1.119911,0.029757,...,0.197215,0.055606,-0.012920,-0.006648,-0.003542,-0.024286,0.022466,0.016447,0.010572,-5.719606e-15
3,1.888773,-0.459735,0.054660,0.092139,0.229733,0.202005,-1.161824,-0.299666,-1.012529,-0.077446,...,0.226854,0.009447,0.068300,-0.051998,-0.016803,-0.015503,0.047927,-0.005671,0.004349,-1.747577e-15
4,2.532461,-0.222875,0.145634,-0.099429,0.454834,0.129949,-1.009218,-0.236410,-0.921815,-0.169922,...,0.116325,0.079064,0.194088,-0.046852,-0.026748,-0.016120,0.003668,-0.013320,-0.000138,3.819945e-15


The numbers are above are more relative to each other, but we still see that year is large, and airports comparatively small

In [845]:
sumExplainedVariance = sum(pca.explained_variance_)
varianceExplained = [(i / sumExplainedVariance)*100 for i in sorted(pca.explained_variance_, reverse=True)]
cumulativeVarianceexplained = np.cumsum(varianceExplained)

In [846]:
varianceExplained

[9.1793104159790371,
 4.5826748168191793,
 2.1218636620993561,
 1.7123176337040837,
 1.5695533451728261,
 1.3231181225323163,
 1.2753729294229348,
 1.2512200550374795,
 1.2087328176470435,
 1.1938048450585947,
 1.1606381007803546,
 1.1540373444346652,
 1.1524899430356821,
 1.1523002451517879,
 1.1522265805260756,
 1.1522265805260747,
 1.1522265805260743,
 1.1522265805260743,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260734,
 1.1522265805260727,
 1.1522265805260727,
 1.1522265805260727,
 1.1522265805260727,
 1.1522265805260727,
 1.1522265805260727,
 1.1522265805260727,
 1.1522265805

In [847]:
cumulativeVarianceexplained

array([   9.17931042,   13.76198523,   15.88384889,   17.59616653,
         19.16571987,   20.488838  ,   21.76421093,   23.01543098,
         24.2241638 ,   25.41796864,   26.57860674,   27.73264409,
         28.88513403,   30.03743428,   31.18966086,   32.34188744,
         33.49411402,   34.6463406 ,   35.79856718,   36.95079376,
         38.10302034,   39.25524692,   40.4074735 ,   41.55970008,
         42.71192666,   43.86415324,   45.01637982,   46.1686064 ,
         47.32083298,   48.47305957,   49.62528615,   50.77751273,
         51.92973931,   53.08196589,   54.23419247,   55.38641905,
         56.53864563,   57.69087221,   58.84309879,   59.99532537,
         61.14755195,   62.29977853,   63.45200511,   64.60423169,
         65.75645827,   66.90868485,   68.06091143,   69.21313801,
         70.3653646 ,   71.51759118,   72.66981776,   73.82204434,
         74.97427092,   76.1264975 ,   77.27872408,   78.43095066,
         79.58317724,   80.73540382,   81.8876304 ,   83.03985

This is a dramatic change from our previous explained variances. Year still provides the most significant explained variance at 9, and the rest of the fields contribute more significant portions. We will remove the airport dummy variables, year, and some summary metrics that dont tell us about what operation contribute to cancellations

### Manual Reduction of Dimensions 

In [848]:
dataSubset = data.iloc[:,7:]
dataSubset.head()

,average_taxi_out_time,average taxi out delay,average airport departure delay,average airborne delay,average taxi in delay,average block delay,average gate arrival delay
0,9.89,2.43,12.10,2.46,0.83,2.55,10.87
1,9.79,2.29,11.20,2.26,0.89,2.34,10.24
2,9.89,2.16,12.33,2.12,0.84,2.66,11.82
3,10.34,2.40,12.95,2.19,1.29,3.06,12.71
4,10.41,2.41,12.32,1.82,1.03,2.79,11.48


In [849]:
dummyColumns = dataSubset.columns

pca = PCA()
ss = StandardScaler()

pipe = Pipeline([('ss', ss),('pca', pca)])

tmp = np.array(dataSubset)
dataReduced = pipe.fit_transform(tmp)
dataReduced = pd.DataFrame(dataReduced,columns=dummyColumns)
dataReduced.head()

,average_taxi_out_time,average taxi out delay,average airport departure delay,average airborne delay,average taxi in delay,average block delay,average gate arrival delay
0,-1.898111,-0.109540,0.314515,-0.009415,-0.107235,0.287551,0.215349
1,-2.198705,-0.348177,0.214677,-0.061941,-0.103867,0.233349,0.252064
2,-1.880518,0.166867,-0.009625,0.017524,-0.033136,0.182616,0.225297
3,-1.293260,0.153883,-0.334754,0.285768,-0.228553,0.139093,0.273822
4,-1.779271,-0.043184,-0.313747,0.097101,0.096770,0.229006,0.185914


In [850]:
sumExplainedVariance = sum(pca.explained_variance_)
varianceExplained = [(i / sumExplainedVariance)*100 for i in sorted(pca.explained_variance_, reverse=True)]
cumulativeVarianceexplained = np.cumsum(varianceExplained)

In [851]:
varianceExplained

[64.559131590159708,
 13.733913274574883,
 9.0431398653155597,
 6.6200959274033382,
 4.0089750291073418,
 1.1124563534676228,
 0.92228795997155633]

In [852]:
cumulativeVarianceexplained

array([  64.55913159,   78.29304486,   87.33618473,   93.95628066,
         97.96525569,   99.07771204,  100.        ])

### Maximum Likelyhood Estimation

In [853]:
everything["year"]

,year,year
0,2004.0,2004
1,2004.0,2005
2,2004.0,2006
3,2004.0,2007
4,2004.0,2008
5,2004.0,2009
6,2004.0,2010
7,2004.0,2011
8,2004.0,2012
9,2004.0,2013


In [855]:
tempYear = everything["year"].iloc[:,0]
print tempYear
del everything["year"]
everything["year"] = tempYear
print len(everything.columns.unique())

0       2004.0
1       2004.0
2       2004.0
3       2004.0
4       2004.0
5       2004.0
6       2004.0
7       2004.0
8       2004.0
9       2004.0
10      2004.0
11      2005.0
12      2005.0
13      2005.0
14      2005.0
15      2005.0
16      2005.0
17      2005.0
18      2005.0
19      2005.0
20      2005.0
21      2005.0
22      2006.0
23      2006.0
24      2006.0
25      2006.0
26      2006.0
27      2006.0
28      2006.0
29      2006.0
         ...  
8759    2012.0
8760    2012.0
8761    2012.0
8762    2012.0
8763    2012.0
8764    2012.0
8765    2012.0
8766    2012.0
8767    2013.0
8768    2013.0
8769    2013.0
8770    2013.0
8771    2013.0
8772    2013.0
8773    2013.0
8774    2013.0
8775    2013.0
8776    2013.0
8777    2013.0
8778    2014.0
8779    2014.0
8780    2014.0
8781    2014.0
8782    2014.0
8783    2014.0
8784    2014.0
8785    2014.0
8786    2014.0
8787    2014.0
8788    2014.0
Name: year, dtype: float64
33


In [856]:
try:
    del everything["index"]
    del everything["airport"]
except:
    "Failed to delete something"

In [857]:
print len(everything.columns.unique())

31


In [872]:
dummyColumns = everything.columns

pca = PCA()
ss = StandardScaler()
mcle = MultiColumnLabelEncoder(columns=dummyColumns)

pipe = Pipeline([('mcle',mcle),('ss', ss),('pca', pca)])
#pipe = Pipeline([('mcle',mcle)])

tmp = np.array(everything)
dataEverything = pipe.fit_transform(tmp)
dataEverything = pd.DataFrame(dataEverything,columns=dummyColumns)
dataEverything.head()

,key,locid,ap_name,alias,facility type,faa region,county,city,state,ap type,...,percent on-time gate arrivals,average_gate_departure_delay,average_taxi_out_time,average taxi out delay,average airport departure delay,average airborne delay,average taxi in delay,average block delay,average gate arrival delay,year
0,3.064296,-1.101848,-3.848937,0.081657,-0.851945,0.656718,0.083009,-0.735949,-1.537336,0.721051,...,0.188846,-0.192990,0.043383,-0.048268,-0.022724,0.006009,0.047566,0.007845,-2.083476e-16,-8.710208e-17
1,3.512118,-1.545834,-3.900367,0.272072,-0.843743,0.697293,0.031349,-0.736768,-1.493581,0.535685,...,0.113389,-0.121689,-0.003340,-0.015137,-0.011755,0.021656,0.038747,0.002478,-5.480884e-16,-2.262505e-16
2,2.994270,-0.528058,-3.866233,-0.221542,-1.026009,0.703887,-0.003501,-0.784161,-1.517212,0.683000,...,0.043902,-0.078040,-0.018404,-0.006932,-0.025790,0.026291,0.044562,-0.019920,-3.119792e-16,-1.257198e-16
3,2.351169,-0.280773,-3.754183,-0.453003,-1.021198,0.314354,-0.261301,-0.900769,-1.779625,0.759633,...,0.168812,-0.024352,-0.062377,0.034935,-0.022339,0.062898,0.028575,-0.008857,-6.907407e-17,-2.375561e-17
4,3.039419,-0.931685,-3.840908,-0.130075,-0.984583,0.517530,-0.218182,-0.827039,-1.618441,0.477366,...,0.282284,-0.147836,-0.031048,-0.023312,-0.036213,0.080742,0.022825,-0.003507,2.082547e-14,-8.203750e-15


In [868]:
sumExplainedVariance = sum(pca.explained_variance_)
varianceExplained = [(i / sumExplainedVariance)*100 for i in sorted(pca.explained_variance_, reverse=True)]
cumulativeVarianceexplained = np.cumsum(varianceExplained)

In [869]:
varianceExplained

[32.537917218108532,
 16.92957747984855,
 10.739803809890281,
 7.6425619126835542,
 5.2949688373111004,
 3.8082033557981778,
 3.5178481542039779,
 3.4578889091336431,
 3.1391799890048548,
 2.6353550128743177,
 1.7599827647757929,
 1.5918971126084784,
 1.4162175750964634,
 1.1608024342124033,
 0.88261666433517472,
 0.76735607744023004,
 0.61013957364105997,
 0.47761645580650997,
 0.47021809957795518,
 0.41898143609600735,
 0.25153755974898384,
 0.21335538554974154,
 0.11044730761626445,
 0.065003774072622977,
 0.048108308150873268,
 0.030389159692540409,
 0.012848646382506015,
 0.0086010812088917737,
 0.00057590513051826041,
 1.9706297009934084e-31,
 1.9706297009934084e-31]

In [870]:
cumulativeVarianceexplained

array([  32.53791722,   49.4674947 ,   60.20729851,   67.84986042,
         73.14482926,   76.95303261,   80.47088077,   83.92876968,
         87.06794967,   89.70330468,   91.46328744,   93.05518456,
         94.47140213,   95.63220457,   96.51482123,   97.28217731,
         97.89231688,   98.36993334,   98.84015144,   99.25913287,
         99.51067043,   99.72402582,   99.83447313,   99.8994769 ,
         99.94758521,   99.97797437,   99.99082301,   99.99942409,
        100.        ,  100.        ,  100.        ])

These explained variances have a better spread. With three principle components, we can cover 87.3 percent of the explained variance. Picking 3 also because we can plot 3 dimension, not 4.

### Lets Start Visualing Our New Data

In [ ]:
# sns.set_style("white")
# sns.set_context('talk')

g = sns.JointGrid(proj2.iloc[:,0], proj2.iloc[:,1], space=0, size=8)
g.plot_marginals(sns.distplot, kde=False, color=".7", bins=30)
g.plot_joint(plt.scatter, color=".5", edgecolor="none", alpha=1)
g.set_axis_labels(xlabel='PC1', ylabel='PC2')

In [ ]:
df = pd.DataFrame(pca2.components_,columns=saveColumns)
sns.heatmap(df.iloc[:,:14])


In [ ]:
joined.iloc[:,0:4].plot.density()

In [ ]:
data.iloc[:,4:8].plot.density()

#### 4.2 Find correlations in the data

In [ ]:
sns.heatmap(data.corr())

#### 4.3 What is the value of understanding correlations before PCA? 

Answer: You can see the correlations between predictors before they get obfuscated by the PCA algorithm

#### 4.4 Validate your findings using statistical analysis

#### 4.5 How can you improve your overall analysis? 

Answer: 

### Part 6: Copy your Database to AWS 

Make sure to properly document all of the features of your dataset

### Bonus: Create a 3-Dimensional Plot of your new dataset with PCA applied

In [ ]:
proj2.head()

In [ ]:
%pylab inline
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

fig = plt.figure(figsize=(16,16))
ax = fig.add_subplot(111, projection='3d')
plt.rcParams['legend.fontsize'] = 10   
ax.plot(proj2.ix[30:50,0], proj2.ix[30:50,1], proj2.ix[30:50,2], 'o', markersize=8, color='blue', alpha=0.5, label='class1')
#ax.plot(proj2.ix[10:50,0], proj2.ix[10:50,1], proj2.ix[10:50,2], '^', markersize=8, alpha=0.5, color='red', label='class2')

plt.title('Samples for class 1 and class 2')
ax.legend(loc='upper right')

plt.show()